<a href="https://colab.research.google.com/github/BatXprO/batxpro.githib.io/blob/main/chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install nltk tensorflow keras

## Import libraries

### Subtask:
Import the installed libraries into the notebook.


**Reasoning**:
Import the necessary libraries as instructed.



In [2]:
import nltk
import tensorflow as tf
from tensorflow import keras

## Load data

### Subtask:
Load the data that the chatbot will use for generating responses.


## Load data retry 1

### Subtask:
Load the data that the chatbot will use for generating responses.


**Reasoning**:
Since the previous attempt to load `intents.json` failed because the file was not found, I need to create a dummy `intents.json` file with some sample data to proceed with the subtask of loading and inspecting the data structure.



In [3]:
import json

# Create a dummy intents.json file
dummy_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "How are you", "Is any one there?"],
            "responses": ["Hello", "Good to see you again", "Hi there, how can I help?"],
            "context_set": ""
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["Sad to see you go", "Talk to you later", "Goodbye!"],
            "context_set": ""
        }
    ]
}

with open('intents.json', 'w') as f:
    json.dump(dummy_data, f, indent=4)

print("Created dummy intents.json")

Created dummy intents.json


**Reasoning**:
Now that a dummy `intents.json` file exists, I can load the data from the file, print its keys, and print the first few entries as requested in the instructions. I will also include error handling for the `FileNotFoundError` in case the file creation failed unexpectedly.



In [4]:
import json

try:
    with open('intents.json', 'r') as f:
        data = json.load(f)

    # Print the keys of the loaded data
    print("Keys of the loaded data:")
    print(data.keys())

    # Print the first few entries of the loaded data
    print("\nFirst few entries of the loaded data:")
    if 'intents' in data and isinstance(data['intents'], list):
        for i, intent in enumerate(data['intents'][:2]): # Print first 2 intents
            print(f"Intent {i+1}:")
            print(intent)
    else:
        print("Data structure does not match expected 'intents' list.")

except FileNotFoundError:
    print("Error: intents.json not found.")
except json.JSONDecodeError:
    print("Error: Could not decode JSON from intents.json")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Keys of the loaded data:
dict_keys(['intents'])

First few entries of the loaded data:
Intent 1:
{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is any one there?'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}
Intent 2:
{'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['Sad to see you go', 'Talk to you later', 'Goodbye!'], 'context_set': ''}


## Preprocess data

### Subtask:
Clean and prepare the data for training the model.


**Reasoning**:
Create lists of unique words and tags, stem the words, sort the lists, and create the training data in bag-of-words format.



In [5]:
import nltk
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import random

nltk.download('punkt')

stemmer = SnowballStemmer("english")

words = []
tags = []
documents = []
ignore_words = ['?', '!']

for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort tags
tags = sorted(list(set(tags)))

print(len(documents), "documents")
print(len(tags), "tags", tags)
print(len(words), "unique stemmed words", words)

# create our training data
training = []
output_empty = [0] * len(tags)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


**Reasoning**:
The previous command failed because the 'punkt_tab' resource was not found. The error message suggests downloading it using `nltk.download('punkt_tab')`. This needs to be done before tokenizing.



In [6]:
import nltk
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import random

# Download the required NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')


stemmer = SnowballStemmer("english")

words = []
tags = []
documents = []
ignore_words = ['?', '!']

for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
    if intent['tag'] not in tags:
        tags.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort tags
tags = sorted(list(set(tags)))

print(len(documents), "documents")
print(len(tags), "tags", tags)
print(len(words), "unique stemmed words", words)

# create our training data
training = []
output_empty = [0] * len(tags)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


6 documents
2 tags ['goodbye', 'greeting']
12 unique stemmed words ['ani', 'are', 'bye', 'goodby', 'hi', 'how', 'is', 'later', 'one', 'see', 'there', 'you']
Training data created


## Build model

### Subtask:
Design and build the chatbot model.


**Reasoning**:
Import the necessary layers and models from tensorflow.keras and define the model architecture.



In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the model architecture
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,050 (39.26 KB)

 Trainable params: 10,050 (39.26 KB)

 Non-trainable params: 0 (0.00 B)

## Train model

### Subtask:
Train the chatbot model on the prepared data.


**Reasoning**:
Train the compiled Keras model using the prepared training data and store the training history.



In [8]:
# Train the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Print a message indicating training completion
print("Model training complete.")

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.1778 - loss: 0.7534
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3556 - loss: 0.7052
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3556 - loss: 0.6697
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8222 - loss: 0.6357
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.6103
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.5838
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.5640
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.5423
Epoch 9/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.5172
Epoch 10/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.4970
Epoch 11/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.4664
Epoch 12/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - lo

## Implement chat functionality

### Subtask:
Create functions for interacting with the chatbot.


**Reasoning**:
Define the functions required for interacting with the chatbot, including cleaning the sentence, creating a bag of words, predicting the class, getting a response, and a main chat function.



In [9]:
import nltk
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import random

stemmer = SnowballStemmer("english")

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if s in ['?','!']: # ignore punctuation
                    bag[i] = 0
    return np.array(bag)

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bag_of_words(sentence, words)
    res = model.predict(np.array([p]), verbose=0)[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": tags[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = get_response(ints, data)
    return res

# Main chat function
print("Go! Type 'quit' to exit")
while True:
    sentence = input("")
    if sentence.lower() == "quit":
        break

    res = chatbot_response(sentence)
    print(res)

Go! Type 'quit' to exit
hi
Hello
how are you today
Good to see you again
quit


## Test chatbot

### Subtask:
Test the chatbot with various inputs to ensure it functions correctly.


**Reasoning**:
Run the chat loop code to test the chatbot with various inputs.



In [10]:
# Main chat function
print("Go! Type 'quit' to exit")
while True:
    sentence = input("")
    if sentence.lower() == "quit":
        break

    res = chatbot_response(sentence)
    print(res)

Go! Type 'quit' to exit
hello
Sad to see you go
why
Talk to you later
okay
Talk to you later
quit
